In [1]:
import pandas as pd
import numpy as np
import scipy as sp

In [11]:
%matplotlib inline

In [18]:
data = [] 
with open('output_results.txt') as f:
    for line in f:
        data.append(line.strip().split(','))
 
data_scrubbed = []
for line in data[2:]:
    fields = [int(a, 2) for a in line]
    data_scrubbed.append(fields)
        
df = pd.DataFrame(data_scrubbed, columns=data[0])
df

,addr_tb,data_tb
0,1,0
1,2,0
2,3,0
3,4,3
4,5,3
...,...,...
499994,8475,3877
499995,8476,3874
499996,8477,3874
499997,8478,3874


In [24]:
df[:1028]

,addr_tb,data_tb
0,1,0
1,2,0
2,3,0
3,4,3
4,5,3
...,...,...
1023,1024,783
1024,1025,783
1025,1026,783
1026,1027,783


In [20]:
print(pd.__version__)

1.2.4
